<a href="https://colab.research.google.com/github/silikmaz/Railimag_nero/blob/master/public_railimag_nero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> installing_Google!</font></b>
 
url = "1" #@param ["1", "2"] {allow-input: true}
 
#@markdown <font color="red" size="+1"><div>
#@markdown if  error at the upload. Please write to vladmaznikovya@mail.ru
#@markdown </div>
#@markdown </font>
from google.colab import files
if url == "1":
  !gdown https://drive.google.com/uc?id=1ZjNexAU65ML-04NBBu3VAeupFzMZB695&export=download
  !unzip Railimag_nero_1.zip
if url == "2":
  !gdown https://drive.google.com/uc?id=1df3zXezKnDPGijQHjaFI_XQummBBRY76&export=download
  !unzip Railimag_nero_1_1.zip
%cd Railimag_nero/

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Let's ROCK!</font></b>
 
#@title Example form fields
#@markdown <div> neural network for internal and demonstrative use. </div>  <div> You can only use it for non commercial purposes. </div> <div>please upload art 16:9</div>
 
model_url = "ARTS" #@param ["Beowulf5K", "ARTS"] {allow-input: true}
Photo_in_Zip = "on" #@param ["yes", "on"] {allow-input: true}
vertical = "720" #@param {type:"string"}
 
 
 
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import functools
import torch.nn as nn
import torch.nn.functional as F
import os
import sys
import time
 
 
 
 
 
 
if Photo_in_Zip == "on":
 %cd input_imag/
 uploaded = files.upload()
 %cd ../
else:
 %cd input_imag/
 print ("upload a zip photo archive")
 uploaded = files.upload()
 %cd ../
 
 drizip = './input_imag/'
 dirs = os.listdir( drizip )
 import zipfile
 %cd input_imag/
 for item in dirs:
  if item.endswith(".zip"):
    z = zipfile.ZipFile(item, 'r')
    z.extractall()
    z.close()
 %cd ../
 
 
 
 
 
 
 
 
 
 
 
def make_layer(block, n_layers):
   layers = []
   for _ in range(n_layers):
       layers.append(block())
   return nn.Sequential(*layers)
 
 
class RailimagBlock_5C(nn.Module):
   def __init__(self, nf=64, gc=32, bias=True):
       super(RailimagBlock_5C, self).__init__()
       # gc: growth channel, i.e. intermediate channels
       self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1, bias=bias)
       self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1, bias=bias)
       self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1, bias=bias)
       self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1, bias=bias)
       self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1, bias=bias)
       self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
 
       # initialization
       # mutil.initialize_weights([self.conv1, self.conv2, self.conv3, self.conv4, self.conv5], 0.1)
 
   def forward(self, x):
       x1 = self.lrelu(self.conv1(x))
       x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
       x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
       x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
       x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
       return x5 * 0.2 + x
 
 
class RRDB(nn.Module):
   '''Residual in Residual Dense Block'''
 
   def __init__(self, nf, gc=32):
       super(RRDB, self).__init__()
       self.RDB1 = RailimagBlock_5C(nf, gc)
       self.RDB2 = RailimagBlock_5C(nf, gc)
       self.RDB3 = RailimagBlock_5C(nf, gc)
 
   def forward(self, x):
       out = self.RDB1(x)
       out = self.RDB2(out)
       out = self.RDB3(out)
       return out * 0.2 + x
 
 
class RRDBNERO(nn.Module):
   def __init__(self, in_nc, out_nc, nf, nb, gc=32):
       super(RRDBNERO, self).__init__()
       RRDB_block_f = functools.partial(RRDB, nf=nf, gc=gc)
 
       self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1, bias=True)
       self.RRDB_trunk = make_layer(RRDB_block_f, nb)
       self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
       #### upsampling
       self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
       self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
       self.HRconv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
       self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1, bias=True)
 
       self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
 
   def forward(self, x):
      fea = self.conv_first(x)
      trunk = self.trunk_conv(self.RRDB_trunk(fea))
      fea = fea + trunk
 
      fea = self.lrelu(self.upconv1(F.interpolate(fea, scale_factor=2, mode='nearest')))
      fea = self.lrelu(self.upconv2(F.interpolate(fea, scale_factor=2, mode='nearest')))
      out = self.conv_last(self.lrelu(self.HRconv(fea)))
 
      return out
 
 
 
 
 
 
 
 
test_img_folder2 = './input_imag/'
test_img_folder1 = './input_imag_1/'
 
from PIL import Image
dirs = os.listdir( test_img_folder2 )
for item in dirs:
  if item.endswith(".zip"):
    os.remove(test_img_folder2+item) 
  if item.endswith(".png") | item.endswith(".jpg"):
   if os.path.isfile(test_img_folder2+'/'+item):
     print (item);
     im = Image.open(test_img_folder2+'/'+item)
     (width, height) = im.size
             
            
             
             
     (width, height) = im.size
     h_x = int(vertical) 
     w_x= width/(height/h_x)
             
     artx = round(w_x)
     arty = round(h_x)
             
     f, e = os.path.splitext(test_img_folder1+item)
     imResize = im.resize((artx,arty), Image.ANTIALIAS)
     imResize.save(f + '.png', 'png', quality=70)
     os.remove(test_img_folder2+item)
     
 
 
 
 
 
 
test_img_folder = './input_imag_1/*'
results_img_folder = './results/'
model_path = './model/'+model_url+'.pth'
device = torch.device('cuda')
 
 
model = RRDBNERO(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)
 
print('Model path {:s}. \nTesting...'.format(model_path))
 
idx = 0
for path in glob.glob(test_img_folder):
   idx += 1
   base = osp.splitext(osp.basename(path))[0]
   print(idx, base)
   # read images
   img = cv2.imread(path, cv2.IMREAD_COLOR)
   img = img * 1.0 / 255
   img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
   img_LR = img.unsqueeze(0)
   img_LR = img_LR.to(device)
   
   with torch.no_grad():
       output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
   output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
   output = (output * 255.0).round()
   cv2.imwrite(results_img_folder+'{:s}_X1.png'.format(base), output)
    
   if Photo_in_Zip == "on":
     try: files.download(results_img_folder+'{:s}_X1.png'.format(base))
     except: raise Exception("No result image")
  
 
if Photo_in_Zip == "yes":   
  %cd results/
  z = zipfile.ZipFile('imag.zip', 'w') 
  for item in os.listdir( "./" ):
    if item.endswith(".png"):
       z.write(item)
  z.close()
  time.sleep(10)
  %cd ../
 
 
 
 
 
 
   
 
    
 
#@markdown ---
#@markdown <Div>
#@markdown Support the "Railimag_nero" project and speed up development:
#@markdown </div> <div> 
#@markdown Sberbank Card - 4276 6719 5951 5831(the of Maznikov Vladislav Andreevich)
#@markdown </div> <div> Yandex.Money -410015679919167</div> <div>  
#@markdown PAYPAL - vladmaznikovya@mail.ru
#@markdown </div> <div> Copyright © maznikov Vladislav </div>
 
#@markdown <font color="red" size="+1">
#@markdown <div color='red'> if an error RuntimeError: CUDA out of memory. Tried to allocate*****   then reduce the vertical.  </div> <div>  don't forget vertical*4  </div></font>

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3">download zip imag </font></b>
 
from google.colab import output
%cd results/
import os
import time
files.download('imag.zip')
files.download('')
 
 
output.clear()
%cd ../
#@markdown <font color="red" size="+1">
#@markdown <div color='red'>
#@markdown <div>the  error " files.download (") " is normal for this block </div> 
#@markdown </font>

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> remove residue </font></b>
if Photo_in_Zip == "yes":
   %cd ../
import os
dirs = os.listdir('./input_imag_1/')
pasfor = './input_imag_1/'
 
for item in dirs:
   dirs1 = pasfor+item
  
   os.remove(pasfor+item)
dirs = os.listdir('./results/')
pasfor = './results/'
 
for item in dirs:
   dirs1 = pasfor+item
  
   os.remove(pasfor+item)